In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [2]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    """
    DeclarativeBase 를 상속 받은 Base 라는 하위 클래스를 만들고 시작.
    이 Base 에 Mapping 된 클래스들은 database 에서 단일 테이블임.

    `__tablename__` 을 클래스 레벨의 속성으로 지녀야 함.
    """
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [4]:
Base.metadata.tables

FacadeDict({'user_account': Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>, nullable=False), Column('fullname', String(), table=<user_account>), schema=None), 'address': Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('email_address', String(), table=<address>, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>, nullable=False), schema=None)})

In [4]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import registry, relationship

# 클래스 정의 (변경 없음)
class User:
    pass

class Address:
    pass

mapper_registry = registry()

user_table = Table(
    "user",
    mapper_registry.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String(30)),
)

address_table = Table(
    "address",
    mapper_registry.metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("user.id")),
    Column("email_address", String(50)),
)

address_mapper = mapper_registry.map_imperatively(Address, address_table)

user_mapper = mapper_registry.map_imperatively(
    User,
    user_table,
    properties={
        "addresses": relationship(Address, backref="user", order_by=address_table.c.id)
    },
)

mapper_registry.metadata.tables

FacadeDict({'user': Table('user', MetaData(), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user>), Column('fullname', String(length=30), table=<user>), schema=None), 'address': Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user.id'), table=<address>), Column('email_address', String(length=50), table=<address>), schema=None)})